In [108]:
using MLJLinearModels
using NCDatasets
using Revise
using HDF5
using RadarQC
using CategoricalArrays

##Train model with DesRosiers and Bell configuration 
NCP, AHT, STD(VV), PGG, RNG  

LoadError: MethodError: no method matching CategoricalValue(::Int64)

[0mClosest candidates are:
[0m  CategoricalValue(::Any, [91m::Union{SubArray{<:Any, N, <:CategoricalArray{T, <:Any, R}} where {T, N, R<:Integer}, CategoricalArray, CategoricalValue}[39m)
[0m[90m   @[39m [35mCategoricalArrays[39m [90m~/.julia/packages/CategoricalArrays/0yLZN/src/[39m[90m[4mvalue.jl:10[24m[39m
[0m  CategoricalValue(::Integer, [91m::CategoricalPool[39m)
[0m[90m   @[39m [35mCategoricalArrays[39m [90m[4mdeprecated.jl:103[24m[39m
[0m  CategoricalValue([91m::CategoricalPool{T, R, CategoricalValue{T, R}}[39m, [91m::R[39m) where {T<:Union{AbstractChar, AbstractString, Number}, R<:Integer}
[0m[90m   @[39m [35mCategoricalArrays[39m [90m~/.julia/packages/CategoricalArrays/0yLZN/src/[39m[90m[4mtypedefs.jl:80[24m[39m
[0m  ...


In [3]:
remove_validation("TRAINING_CENTER.h5"; training_output = "training_center_no_validation.h5", validation_output = "valdiation_center.h5" )

standardize (generic function with 1 method)

In [93]:
MLJ.table(features)

Tables.MatrixTable{Matrix{Float64}} with 3756795 rows, 5 columns, and schema:
 :x1  Float64
 :x2  Float64
 :x3  Float64
 :x4  Float64
 :x5  Float64

In [58]:
validation_data = h5open("../valdiation_center.h5")
features = validation_data["X"][:,:]
###Flatten targets 
targets = validation_data["Y"][:,:][:]
close(validation_data)

###Standardize features to aid in regression convergence 
features = mapslices(standardize, features, dims=1)

3756795×5 Matrix{Float64}:
 0.0182841   0.0590536  0.232203   0.0  2.78346e-5
 0.0942335   0.120031   0.0120462  0.0  0.0209038
 0.0942335   0.18101    0.0289368  0.0  0.0417797
 0.0787623   0.241991   0.0717943  0.0  0.0626557
 0.0         0.302975   0.21883    0.0  0.0835317
 0.00140647  0.839726   0.694006   0.0  0.26724
 0.105485    0.10781    0.0118779  0.0  0.0167286
 0.104079    0.168757   0.0194843  0.0  0.0376046
 0.0914205   0.229706   0.0611143  0.0  0.0584805
 0.0421941   0.290658   0.0965469  0.0  0.0793565
 0.0618847   0.0712291  0.178664   0.0  0.00420303
 0.102672    0.132108   0.016209   0.0  0.025079
 0.0886076   0.192989   0.0206875  0.0  0.0459549
 ⋮                                      
 0.00562588  0.553636   0.280488   0.0  0.144072
 0.00562588  0.222238   0.468663   0.0  0.0334294
 0.00281294  0.561346   0.242336   0.0  0.14616
 0.0900141   0.128065   0.243299   0.0  0.00211543
 0.00281294  0.549912   0.346287   0.0  0.141984
 0.00281294  0.147003   0.587957   0

In [110]:
targets = categorical(targets)

3756795-element CategoricalArray{Int64,1,UInt32}:
 1
 1
 1
 1
 0
 0
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [ ]:
names = ["OK", "OK1", "OK2", "OK3", "OK4"]
new_features = ()
for name in names:
    push!(new_features, )

In [61]:
###Now, let's train a logistic classifier on this validation set to determine parameter importance 
###Utilize the L1/LASSO penalty as we're looking to minimize Feature coefficients 
λs = 10. .^ (-1:-1:-5)
models = [] 
for λ in λs 
    model = LassoRegression(λ)
    fit(model, features, targets)
    push!(models, model) 
end

In [ ]:
@load LogisticClassifier pkg=MLJLinearModels



function standardize(column)
    col_max = maximum(column)
    col_min = minimum(column)
    return (map(x-> (x - col_min) / (col_max - col_min), column))
end 


##Takes in a vector of λ penalty values, 
##and input file. Opens input HDF5 file, and
##trains a series of Lasso regression models on it
##eventually returning a matrix of parameter values 
function get_feature_importance(input_file_path, λs)

    training_data = h5open(input_file_path)
    
    ###Standardize features to expedite regression convergence 
    features = mapslices(standardize, training_data["X"][:,:], dims=1)
    ###Flatten targets and convert to categorical datatime 
    targets = categorical(training_data["Y"][:,:][:])
    params = attrs(validation_data)["Parameters"]

    close(validation_data)

    coef_values = Dict(param => [] for param in params)
    coef_values["λ"] = λs 

    for λ in λs

        mdl = LogisticClassifier(;lambda=λ, penalty=:l1)
        mach = machine(mdl, MLJ.table(features), targets[:])
        coefs = fitted_params(mach).coefs 

        for i, param in enumerate(parmas)
            push!(coef_values[param], coefs[i])
        end 
    end 

    return coef_values 
end 


In [150]:
curr["λ"] = []


Any[]

In [151]:
curr

Dict{String, Vector{Any}} with 3 entries:
  "OK"  => [2.5, 2.5, 2.5, 2.5]
  "OK2" => []
  "λ"   => []

In [147]:
push!(curr["OK"], 2.5)

4-element Vector{Any}:
 2.5
 2.5
 2.5
 2.5

In [148]:
curr

Dict{String, Vector{Any}} with 2 entries:
  "OK"  => [2.5, 2.5, 2.5, 2.5]
  "OK2" => []

import MLJLinearModels ✔


[ Info: For silent loading, specify `verbosity=0`. 


untrained Machine; caches model-specific representations of data
  model: LogisticClassifier(lambda = 0.001, …)
  args: 
    1:	Source @943 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @553 ⏎ AbstractVector{Multiclass{2}}


In [120]:
fit!(mach)

[ Info: Training machine(LogisticClassifier(lambda = 0.001, …), …).
┌ Info: Solver: ProxGrad
│   accel: Bool true
│   max_iter: Int64 1000
│   tol: Float64 0.0001
│   max_inner: Int64 100
│   beta: Float64 0.8
└   gram: Bool false


trained Machine; caches model-specific representations of data
  model: LogisticClassifier(lambda = 0.001, …)
  args: 
    1:	Source @943 ⏎ Table{AbstractVector{Continuous}}
    2:	Source @553 ⏎ AbstractVector{Multiclass{2}}


In [121]:
predict(mach, features)

3756795-element CategoricalDistributions.UnivariateFiniteVector{Multiclass{2}, Int64, UInt32, Float64}:
 UnivariateFinite{Multiclass{2}}(0=>0.838, 1=>0.162)
 UnivariateFinite{Multiclass{2}}(0=>0.0921, 1=>0.908)
 UnivariateFinite{Multiclass{2}}(0=>0.0861, 1=>0.914)
 UnivariateFinite{Multiclass{2}}(0=>0.128, 1=>0.872)
 UnivariateFinite{Multiclass{2}}(0=>0.654, 1=>0.346)
 UnivariateFinite{Multiclass{2}}(0=>0.979, 1=>0.0208)
 UnivariateFinite{Multiclass{2}}(0=>0.084, 1=>0.916)
 UnivariateFinite{Multiclass{2}}(0=>0.0722, 1=>0.928)
 UnivariateFinite{Multiclass{2}}(0=>0.103, 1=>0.897)
 UnivariateFinite{Multiclass{2}}(0=>0.207, 1=>0.793)
 UnivariateFinite{Multiclass{2}}(0=>0.588, 1=>0.412)
 UnivariateFinite{Multiclass{2}}(0=>0.0825, 1=>0.917)
 UnivariateFinite{Multiclass{2}}(0=>0.0803, 1=>0.92)
 ⋮
 UnivariateFinite{Multiclass{2}}(0=>0.551, 1=>0.449)
 UnivariateFinite{Multiclass{2}}(0=>0.981, 1=>0.0192)
 UnivariateFinite{Multiclass{2}}(0=>0.439, 1=>0.561)
 UnivariateFinite{Multiclass{2}}(0=>0.6

In [127]:
params = fitted_params(mach)

(classes = CategoricalValue{Int64, UInt32}[0, 1],
 coefs = [:x1 => 13.65820984546808, :x2 => 3.9062392177270024, :x3 => -11.902021767020244, :x4 => -4.3657096152449935, :x5 => 1.767711557646921],
 intercept = 0.6385524768597355,)

In [136]:
var = "x1"
params.coefs

5-element Vector{Pair{Symbol, Float64}}:
 :x1 => 13.65820984546808
 :x2 => 3.9062392177270024
 :x3 => -11.902021767020244
 :x4 => -4.3657096152449935
 :x5 => 1.767711557646921

In [ ]:
train, test = partition(eachindex(targets), .7)

In [117]:
targets

3756795-element CategoricalArray{Int64,1,UInt32}:
 1
 1
 1
 1
 0
 0
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [90]:
typeof(X)

@NamedTuple{Crim::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Zn::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Indus::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, NOx::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Rm::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Age::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Dis::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Rad::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Tax::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, PTRatio::SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Black::SubArray{Float64, 1, Matri

In [75]:
fieldnames(models[1])

LoadError: MethodError: no method matching fieldnames(::GeneralizedLinearRegression{L2Loss, ScaledPenalty{L1Penalty}})

[0mClosest candidates are:
[0m  fieldnames([91m::Core.TypeofBottom[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mreflection.jl:170[24m[39m
[0m  fieldnames([91m::Type{<:Tuple}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mreflection.jl:172[24m[39m
[0m  fieldnames([91m::UnionAll[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mreflection.jl:169[24m[39m
[0m  ...


In [70]:
objective(models[1], features, targets)

#130 (generic function with 1 method)

In [69]:
machine(LassoRegression(.5), features, targets)

LoadError: MethodError: no method matching machine(::GeneralizedLinearRegression{L2Loss, ScaledPenalty{L1Penalty}}, ::Matrix{Float64}, ::Vector{Int64})

[0mClosest candidates are:
[0m  machine([91m::Type{<:Model}[39m, ::Any...; kwargs...)
[0m[90m   @[39m [33mMLJBase[39m [90m~/.julia/packages/MLJBase/mIaqI/src/[39m[90m[4mmachines.jl:336[24m[39m
[0m  machine([91m::Static[39m, ::Any...; cache, kwargs...)
[0m[90m   @[39m [33mMLJBase[39m [90m~/.julia/packages/MLJBase/mIaqI/src/[39m[90m[4mmachines.jl:340[24m[39m
[0m  machine([91m::Union{Symbol, Model}[39m, ::Any, ::Any...; scitype_check_level, kwargs...)
[0m[90m   @[39m [33mMLJBase[39m [90m~/.julia/packages/MLJBase/mIaqI/src/[39m[90m[4mmachines.jl:364[24m[39m
[0m  ...


In [65]:
predict(models[1], features)

LoadError: MethodError: no method matching predict(::GeneralizedLinearRegression{L2Loss, ScaledPenalty{L1Penalty}}, ::Matrix{Float64})

[0mClosest candidates are:
[0m  predict([91m::MLJModels.DeterministicConstantRegressor[39m, ::Any, [91m::Any[39m)
[0m[90m   @[39m [36mMLJModels[39m [90m~/.julia/packages/MLJModels/iFOi9/src/builtins/[39m[90m[4mConstant.jl:42[24m[39m
[0m  predict([91m::ConstantClassifier[39m, ::Any, [91m::Any[39m)
[0m[90m   @[39m [36mMLJModels[39m [90m~/.julia/packages/MLJModels/iFOi9/src/builtins/[39m[90m[4mConstant.jl:68[24m[39m
[0m  predict([91m::MLJModels.DeterministicConstantClassifier[39m, ::Any, [91m::Any[39m)
[0m[90m   @[39m [36mMLJModels[39m [90m~/.julia/packages/MLJModels/iFOi9/src/builtins/[39m[90m[4mConstant.jl:90[24m[39m
[0m  ...


In [39]:
using MLJ
schema(features)

LoadError: ArgumentError: Cannot inspect the internal scitypes of a non-tabular object. 

In [2]:
using RadarQC

Precompiling RadarQC
        Info Given RadarQC was explicitly requested, output will be shown live 
    CondaPkg Found dependencies: /Users/ischluesche/Documents/Grad_School/Research/RadarQC/CondaPkg.toml
    CondaPkg Found dependencies: /Users/ischluesche/.julia/packages/PythonCall/wXfah/CondaPkg.toml
    CondaPkg Dependencies already up to date
  ✓ RadarQC
  1 dependency successfully precompiled in 16 seconds. 313 already precompiled.
  1 dependency had output during precompilation:
┌ RadarQC
│  [Output was shown above]
└  


In [3]:
###Make sure to use absolute paths here 
CASE_PATHS= ["/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/BAMEX", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/HAGUPIT", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/RITA", 
             "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/VORTEX"]

TRAINING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TRAINING"
TESTING_PATH = "/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

"/Users/ischluesche/Documents/Grad_School/Research/RadarQC/CFRADIALS/CASES/TESTING"

In [5]:
split_training_testing!(CASE_PATHS, TRAINING_PATH, TESTING_PATH)


TOTAL NUMBER OF TDR SCANS ACROSS ALL CASES: 1780
TESTING SCANS PER CASE 89
NUMBER OF SCANS IN CASE: 482
TRAINING GROUP SIZE: 196 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 228 ASSIGNED TRAINING
 INDEXES 229 TO 257 ASSIGNED TESTING
 INDEXES 258 TO 453 ASSIGNED TRAINING
 INDEXES 454 TO 482 ASSIGNED TESTING
Total length of case files: 482
Length of testing files: 89 - 0.18464730290456433 percent
Length of testing_files: 393 - 0.8153526970954357 percent
NUMBER OF SCANS IN CASE: 238
TRAINING GROUP SIZE: 74 + REMAINDER: 1
TESTING GROUP SIZE: 29 + REMAINDER 2

 INDEXES 1 TO 31 ASSIGNED TESTING
 INDEXES 31  TO 106 ASSIGNED TRAINING
 INDEXES 107 TO 135 ASSIGNED TESTING
 INDEXES 136 TO 209 ASSIGNED TRAINING
 INDEXES 210 TO 238 ASSIGNED TESTING
Total length of case files: 238
Length of testing files: 89 - 0.3739495798319328 percent
Length of testing_files: 149 - 0.6260504201680672 percent
NUMBER OF SCANS IN CASE: 794
TRAINING GROUP SIZE

In [8]:
calculate_features("./CFRADIALS/CASES/TRAINING", "./MODELS/DesRosiers_Bell_23/config.txt", "training_features.h5"; 
                    verbose=true, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
                    REMOVE_HIGH_PGG = true, remove_variable = "VV" )

calculate_features("./CFRADIALS/CASES/TESTING", "./MODELS/DesRosiers_Bell_23/config.txt", "testing_features.h5"; 
                    verbose=false, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
                    REMOVE_HIGH_PGG = true, remove_variable = "VV")

LoadError: UndefVarError: `info` not defined

In [3]:
RadarQC.get_task_params("./sample_config.txt", ["DBZ", "VEL", "VV", "VG", "NCP"])

5-element Vector{String}:
 "NCP"
 "AHT"
 "STD(VV)"
 "PGG"
 "RNG"

In [5]:
parentmodule(get_task_params)

RadarQC

In [5]:
using NCDatasets

In [11]:
data_path = "./cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR (1).nc"
currset = Dataset("./cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR (1).nc")

Dataset: ./cfrad.20220918_124128.197_to_20220918_124132.176_N42RF-TS_AIR (1).nc
Group: /

Dimensions
   time = 360
   range = 721
   sweep = 1
   string_length_8 = 8
   string_length_32 = 32
   status_xml_length = 1
   r_calib = 1
   frequency = 1

Variables
  volume_number  
    Attributes:
     long_name            = data_volume_index_number
     units                = 
     _FillValue           = -9999

  platform_type   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = platform_type
     options              = fixed, vehicle, ship, aircraft_fore, aircraft_aft, aircraft_tail, aircraft_belly, aircraft_roof, aircraft_nose, satellite_orbit, satellite_geostat

  primary_axis   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = primary_axis_of_rotation
     options              = axis_z, axis_y, axis_x, axis_z_prime, axis_y_prime, axis_x_prime

  status_xml   (1)
   

In [14]:
weight_matrix = [ones(7,7)]
calculate_features(data_path, ["STD(VG)"], weight_matrix, "garbage.out"; remove_variable = "VG")

OUTPUTTING DATA IN HDF5 FORMAT TO FILE: garbage.out
COMPLETED PROCESSING 1 FILES IN 0.43 SECONDS

WRITING DATA TO FILE OF SHAPE (59749, 1)
X TYPE: Matrix{Float64}


In [9]:
func_regex::Regex = r"(\w{1,})\((\w{1,})\)"


r"(\w{1,})\((\w{1,})\)"

In [13]:
retval = match(func_regex, "STD(VV)")

RegexMatch("STD(VV)", 1="STD", 2="VV")

In [14]:
retval[1]

"STD"

In [3]:
#Train model with DesRosiers and Bell configuration 
using Missings 
using RadarQC

vars = ["NCP", "AHT", "STD(VV)", "PGG", "RNG"] 
empty_matrix = [1. 1. ; 1. 0.]
std_matrix = allowmissing(ones(5,5))
std_matrix_center = allowmissing(ones(5,5))

std_matrix[3,3] = missing 

weights_missing = [empty_matrix, empty_matrix, std_matrix, empty_matrix, empty_matrix]
weights_center = [empty_matrix, empty_matrix, std_matrix_center, empty_matrix, empty_matrix]

calculate_features("../CFRADIALS/CASES/TRAINING", vars, weights_missing, "TRAINING_MISSING_CENTER.h5", 
verbose=true, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
REMOVE_HIGH_PGG = true, remove_variable = "VV" )




OUTPUTTING DATA IN HDF5 FORMAT TO FILE: TRAINING_MISSING_CENTER.h5
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222027.272_to_19950516_222030.101_TA-ELDR_AIR.nc in 2.6925580501556396 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222028.676_to_19950516_222031.494_TF-ELDR_AIR.nc in 0.12719297409057617 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222030.112_to_19950516_222032.931_TA-ELDR_AIR.nc in 0.11799001693725586 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222031.505_to_19950516_222034.335_TF-ELDR_AIR.nc in 0.12135601043701172 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222032.942_to_19950516_222035.760_TA-ELDR_AIR.nc in 0.11592602729797363 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222034.346_to_19950516_222037.164_TF-ELDR_AIR.nc in 0.15608716011047363 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222035.771_to_19950516_222038.601_TA-ELDR_AIR.nc in 0.46396684646606445 seconds
Pro

POSSIBLE ERRONEOUS CFRAD DIMENSIONS... SKIPPING ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180849.988_to_20050922_180849.988_TF-ELDR_AIR.nc


Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180851.647_to_20050922_180854.976_TA-ELDR_AIR.nc in 0.14768576622009277 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180853.317_to_20050922_180856.646_TF-ELDR_AIR.nc in 0.18037819862365723 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180854.988_to_20050922_180858.304_TA-ELDR_AIR.nc in 0.22566604614257812 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180856.659_to_20050922_180859.963_TF-ELDR_AIR.nc in 0.44680309295654297 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180858.317_to_20050922_180901.621_TA-ELDR_AIR.nc in 0.13226914405822754 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180859.975_to_20050922_180903.292_TF-ELDR_AIR.nc in 0.12027692794799805 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180901.633_to_20050922_180904.950_TA-ELDR_AIR.nc in 0.29289793968200684 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180903.304_to_20

In [5]:
calculate_features("../CFRADIALS/CASES/TRAINING", vars, weights_center, "TRAINING_CENTER.h5", 
verbose=true, HAS_MANUAL_QC = true, REMOVE_LOW_NCP = true, 
                    REMOVE_HIGH_PGG = true, remove_variable = "VV" )

OUTPUTTING DATA IN HDF5 FORMAT TO FILE: TRAINING_CENTER.h5
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222027.272_to_19950516_222030.101_TA-ELDR_AIR.nc in 0.44043803215026855 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222028.676_to_19950516_222031.494_TF-ELDR_AIR.nc in 0.12883806228637695 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222030.112_to_19950516_222032.931_TA-ELDR_AIR.nc in 0.11053109169006348 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222031.505_to_19950516_222034.335_TF-ELDR_AIR.nc in 0.1160728931427002 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222032.942_to_19950516_222035.760_TA-ELDR_AIR.nc in 0.11437702178955078 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222034.346_to_19950516_222037.164_TF-ELDR_AIR.nc in 1.248013973236084 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.19950516_222035.771_to_19950516_222038.601_TA-ELDR_AIR.nc in 0.12062597274780273 seconds
Processed ../

POSSIBLE ERRONEOUS CFRAD DIMENSIONS... SKIPPING ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180849.988_to_20050922_180849.988_TF-ELDR_AIR.nc


Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180851.647_to_20050922_180854.976_TA-ELDR_AIR.nc in 0.11263108253479004 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180853.317_to_20050922_180856.646_TF-ELDR_AIR.nc in 0.1472628116607666 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180854.988_to_20050922_180858.304_TA-ELDR_AIR.nc in 0.1940469741821289 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180856.659_to_20050922_180859.963_TF-ELDR_AIR.nc in 0.4715869426727295 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180858.317_to_20050922_180901.621_TA-ELDR_AIR.nc in 0.11745309829711914 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180859.975_to_20050922_180903.292_TF-ELDR_AIR.nc in 0.13077378273010254 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180901.633_to_20050922_180904.950_TA-ELDR_AIR.nc in 0.29035282135009766 seconds
Processed ../CFRADIALS/CASES/TRAINING/cfrad.20050922_180903.304_to_20050

In [8]:
###Now, remove validation from each 
remove_validation("./TRAINING_MISSING_CENTER.h5"; training_output= "missing_center_training.h5", validation_output = "missing_center_validation.h5")

LoadError: BoundsError: attempt to access 1423×1 Matrix{Int64} at index [68681866-element BitVector, 1:1]

In [10]:
using HDF5
currset = h5open("TRAINING_MISSING_CENTER.h5")

🗂️ HDF5.File: (read-only) TRAINING_MISSING_CENTER.h5
├─ 🏷️ MISSING_FILL_VALUE
├─ 🏷️ Parameters
├─ 🔢 X
└─ 🔢 Y

In [13]:
size(currset["X"])

(68681866, 5)

In [15]:
currset["Y"][:,:]

1423×1 Matrix{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [ ]:
n = 500
p = 5
X = randn(n, p)
y = 2 * (rand(n) .< 0.5) .- 1   # entries are +-1
λ = 0.5
logistic = LogisticRegression(λ)
theta = fit(logistic, X, y)


In [ ]:
###Just protyping here 

##Will complete a series of logistic regressions, varying lambda values, to determine 
###The most important features passed to 
function tune_model("")